# WaterTAP Workshop 2024: Reverse Osmosis Parameter Fitting Demo

### Today's demonstration will show how to use WaterTAP to determine membrane properties from experimental data and a spec sheets
* #### Part 1: Basic demonstration of Degrees of Freedom (DOF) and solving for known/assumed variables
* #### Part 2: Estimate membrane parameters from spec sheet data
* #### Part 3: Estimate membrane parameters in more complex systems

## Part 0: Import necessary libraries

In [1]:
from pyomo.environ import (
    check_optimal_termination,
    ConcreteModel,
    Constraint,
    value,
    Var,
    NonNegativeReals,
    assert_optimal_termination,
    units as pyunits,
)
from idaes.core import FlowsheetBlock
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from watertap.unit_models.reverse_osmosis_1D import (ReverseOsmosis1D, ConcentrationPolarizationType, 
                                                     MassTransferCoefficient, PressureChangeType)
from watertap.unit_models.reverse_osmosis_0D import (ReverseOsmosis0D,
        ConcentrationPolarizationType, MassTransferCoefficient)
from watertap.core.util.model_diagnostics.infeasible import *
from watertap.core.util.initialization import *
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.solvers import get_solver

## Part 1: Basic demonstration of Degrees of Freedom (DOF) and solving for known/assumed variables
* Create a simple RO unit model
* Define some assumed operating conditions and membrane properties
* Unfix membrane properties and fix performance variables (permeate flow rate, permeate salinity, pressure loss)

<p style="text-align: center"><img src="assets/RO_Stage.png" width="50%">

#### Start building the RO model
#### Define ConcreteModel, FlowsheetBlock, and Property Package

In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

#### Define the RO unit model

In [3]:
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    has_full_reporting = True,
    )

## Degrees of Freedom

#### 4-DOF: Inlet feed state variables (i.e. temperature, pressure, component flowrates)
* Temperature
* Pressure
* H2O Mass Flow Rate
* NaCl Mass Flow Rate

#### 4-DOF: The RO model has at least 4 degrees of freedom that should be fixed for the unit to be fully specified.
* membrane water permeability, A
* membrane salt permeability, B
* permeate pressure
* membrane area

#### 3-DOF: If configuring the RO unit to calculate concentration polarization effects, mass transfer coefficient, and pressure drop, 3 additional degrees of freedom are required
* feed-spacer porosity
* feed-channel height
* membrane length or membrane width or inlet Reynolds number

In [4]:
print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 11


## Specify values for system variables

In [5]:
# fix the 4 inlet state variables
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'NaCl'].fix(0.035)     # feed mass flowrate of TDS (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(0.965)      # feed mass flowrate of water (kg/s)
m.fs.RO.inlet.pressure[0].fix(50E5)                                 # feed pressure (Pa)
m.fs.RO.inlet.temperature[0].fix(298)                               # feed temperature (K)

# fix 2 membrane properties
m.fs.RO.A_comp.fix(4.2e-12)                                         # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)                                          # membrane salt permeability coeff (m/s)

# fix 4 module specficiations
m.fs.RO.area.fix(50)                                                # membrane stage area (m^2)
m.fs.RO.width.fix(5)                                                # membrane stage width (m)
m.fs.RO.feed_side.channel_height.fix(1E-3)                          # channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)                         # spacer porosity in membrane stage (-)

# 1 outlet state variable
m.fs.RO.permeate.pressure[0].fix(101325)                            # permeate pressure (Pa)

In [6]:
print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 0


## Scale all variables

In [7]:
# Set scaling factors for component mass flowrates.
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'NaCl'))

# Set scaling factor for membrane area.
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate scaling factors for all other variables.
calculate_scaling_factors(m)

## Initialize the model

In [8]:
m.fs.RO.initialize()

2024-05-17 15:28:11 [INFO] idaes.init.fs.RO.feed_side: Initialization Complete
2024-05-17 15:28:11 [INFO] idaes.init.fs.RO: Initialization Complete: optimal - Optimal Solution Found


## Setup a solver and run a simulation

In [9]:
# Solve the RO Unit
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

In [10]:
m.fs.RO.report()


Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                            : Value      : Units                 : Fixed : Bounds
                                Hydraulic Diameter :  0.0017321 :                 meter : False : (0.0001, 0.005)
                                     Membrane Area :     50.000 :            meter ** 2 :  True : (0.1, 100000.0)
                                   Membrane Length :     10.000 :                 meter : False : (0.1, 500.0)
                                    Membrane Width :     5.0000 :                 meter :  True : (0.1, 1000.0)
                    NaCl Concentration @Inlet,Bulk :     35.751 : kilogram / meter ** 3 : False : (0.001, 2000.0)
     NaCl Concentration @Inlet,Membrane-Interface  :     42.924 : kilogram / meter ** 3 : False : (0.001, 2000.0)
      

## Now that we have a model that will initialize and solve, let's start exploring the DOF

In [11]:
# We can start by unfixing the water permeability coefficient
m.fs.RO.A_comp.unfix()
print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 1


In [12]:
# And then we can define a fixed value for the permeate flowrate
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp['Liq', 'H2O'].fix(0.4)
print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 0


In [13]:
# Initialize and solve the model given the new fixed value
m.fs.RO.initialize()
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

2024-05-17 15:28:12 [INFO] idaes.init.fs.RO.feed_side: Initialization Complete
2024-05-17 15:28:12 [INFO] idaes.init.fs.RO: Initialization Complete: optimal - Optimal Solution Found


In [14]:
m.fs.RO.report()
m.fs.RO.A_comp.display()


Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                            : Value      : Units                 : Fixed : Bounds
                                Hydraulic Diameter :  0.0017321 :                 meter : False : (0.0001, 0.005)
                                     Membrane Area :     50.000 :            meter ** 2 :  True : (0.1, 100000.0)
                                   Membrane Length :     10.000 :                 meter : False : (0.1, 500.0)
                                    Membrane Width :     5.0000 :                 meter :  True : (0.1, 1000.0)
                    NaCl Concentration @Inlet,Bulk :     35.751 : kilogram / meter ** 3 : False : (0.001, 2000.0)
     NaCl Concentration @Inlet,Membrane-Interface  :     56.315 : kilogram / meter ** 3 : False : (0.001, 2000.0)
      

#### We can see that the water permeability (A_comp) changed from the previous assumed value of 4.2E-12 to achieve the new permeate flow rate

In [15]:
# Similarly, we can also unfix the water permeate flowrate and fix the recovery to 50%
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp['Liq', 'H2O'].unfix()
m.fs.RO.recovery_vol_phase[0.0,'Liq'].fix(0.5)

print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 0


In [16]:
solver = get_solver()
simulation_results = solver.solve(m)
assert_optimal_termination(simulation_results)

In [17]:
m.fs.RO.report()
m.fs.RO.A_comp.display()


Unit : fs.RO                                                               Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                            : Value      : Units                 : Fixed : Bounds
                                Hydraulic Diameter :  0.0017321 :                 meter : False : (0.0001, 0.005)
                                     Membrane Area :     50.000 :            meter ** 2 :  True : (0.1, 100000.0)
                                   Membrane Length :     10.000 :                 meter : False : (0.1, 500.0)
                                    Membrane Width :     5.0000 :                 meter :  True : (0.1, 1000.0)
                    NaCl Concentration @Inlet,Bulk :     35.751 : kilogram / meter ** 3 : False : (0.001, 2000.0)
     NaCl Concentration @Inlet,Membrane-Interface  :     59.914 : kilogram / meter ** 3 : False : (0.001, 2000.0)
      

#### We can see that the A_comp value is adjusted to meet the new recovery



####


## Part 2: Estimating membrane properties given a RO spec sheet
* Identify spec sheet data required to fit membrane properties
* Setup RO unit model to reflect spec sheet system
* Initialize

<p style="text-align: center"><img src="assets/BW30.png" height="480">

In [18]:
permeate_flow = (48 * pyunits.m**3/pyunits.day) * (997.0 * pyunits.kg/pyunits.m**3) # Volumetric flowrate * density to get mass flowrate
recovery = 0.15
feed_flow = permeate_flow / recovery
feed_conc = 2000 * pyunits.mg/pyunits.kg
permeate_conc = feed_conc * (1-0.997)

In [19]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis1D(
        property_package=m.fs.properties,
        has_pressure_change=True,
        pressure_change_type=PressureChangeType.calculated,
        mass_transfer_coefficient=MassTransferCoefficient.calculated,
        concentration_polarization_type=ConcentrationPolarizationType.calculated,
        transformation_scheme="BACKWARD",
        transformation_method="dae.finite_difference",
        finite_elements=10,
        has_full_reporting = True,
        )

In [20]:
print('Degrees of Freedom:', degrees_of_freedom(m.fs.RO))

Degrees of Freedom: 11


## Specify values for system variables

In [21]:
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'NaCl'].fix(feed_flow*feed_conc)
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(feed_flow)
m.fs.RO.inlet.pressure[0].fix(15.5 * pyunits.bar) 
m.fs.RO.inlet.temperature[0].fix(298.15)

m.fs.RO.area.fix(41)
m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(3.5e-8)

m.fs.RO.permeate.pressure[0].fix(101325)
m.fs.RO.feed_side.channel_height.fix(1e-3)
m.fs.RO.feed_side.spacer_porosity.fix(0.95)     
m.fs.RO.length.fix(1.016)

print("DOF = ", degrees_of_freedom(m))
print("RO DOF = ", degrees_of_freedom(m.fs.RO))
assert_no_degrees_of_freedom(m)

DOF =  0
RO DOF =  0


## Scale all variables.

In [22]:
set_scaling_factor(m.fs.RO.area, 1e-3)
set_scaling_factor(m.fs.RO.feed_side.area, 1e-3)
set_scaling_factor(m.fs.RO.width, 1e-2)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))

calculate_scaling_factors(m) 

## Note: 
This salinity given in the spec sheet is low (2g/L). Some of the variables in the WaterTAP RO Unit are scaled and constrained to limits more relavent to seawater concentrations. Sometimes adjusting the bounds on these variables is required so solve in these different conditions

In [23]:
# Release constraints related to low concentrations
for item in [m.fs.RO.permeate_side, m.fs.RO.feed_side.properties_interface]:
    for idx, param in item.items():
        if idx[1] > 0:
            param.molality_phase_comp["Liq", "NaCl"].setlb(1e-5)
            param.pressure_osm_phase["Liq"].setlb(100)

In [24]:
def solve(m, raise_on_failure=True):
    # ---solving---
    solver = get_solver()

    print("\n--------- SOLVING ---------\n")
    results = solver.solve(m)

    if check_optimal_termination(results):
        print("\n--------- OPTIMAL SOLVE!!! ---------\n")

        print(f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}')
        print(f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}')
        print(f'{"Porosity":<20s}{value(m.fs.RO.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO.feed_side.spacer_porosity)):<10s}')
        print('\n')

        return results
    assert False

In [25]:
m.fs.RO.initialize()
results = solve(m)

2024-05-17 15:28:12 [INFO] idaes.init.fs.RO.feed_side: Initialization Complete
2024-05-17 15:28:13 [INFO] idaes.init.fs.RO: Initialization Complete: optimal - Optimal Solution Found

--------- SOLVING ---------


--------- OPTIMAL SOLVE!!! ---------

Water Perm          1.512     l/bar/h/m**2
Salt Perm           0.126     l/h/m**2  
Porosity            0.950     dimensionless




#### Solve for A

In [26]:
# Unfix A variable
m.fs.RO.A_comp.unfix()

# Fix the permeate flow
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp['Liq', 'H2O'].fix(permeate_flow)

print("DOF = ", degrees_of_freedom(m))

DOF =  0


In [27]:
results = solve(m)


--------- SOLVING ---------


--------- OPTIMAL SOLVE!!! ---------

Water Perm          4.171     l/bar/h/m**2
Salt Perm           0.126     l/h/m**2  
Porosity            0.950     dimensionless




#### Solve for B

In [28]:
# Unfix B variable
m.fs.RO.B_comp.unfix()

# Fix the permeate concentration
m.fs.RO.mixed_permeate[0.0].conc_mass_phase_comp['Liq', 'NaCl'].fix((1-0.997)*(2000 * pyunits.mg/pyunits.L))
print("DOF = ", degrees_of_freedom(m))

DOF =  0


In [29]:
results = solve(m, raise_on_failure=True)


--------- SOLVING ---------


--------- OPTIMAL SOLVE!!! ---------

Water Perm          4.173     l/bar/h/m**2
Salt Perm           0.082     l/h/m**2  
Porosity            0.950     dimensionless




## Solve for Pressure Loss and Spacer Porosity

<p style="text-align: center"><img src="assets/BW30_2.png" width="80%">

In [30]:
feed_flow = (19 * pyunits.m**3/pyunits.hr) * (997.0 * pyunits.kg/pyunits.m**3) # Volumetric flowrate * density to get mass flowrate
pressure_loss = -1 * pyunits.bar

## Unfix the performance variables (permeate production and concentration) and fix the solved A & B parameters

In [31]:
# Unfix the permeate flow rate and concentration
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp['Liq', 'H2O'].unfix()
m.fs.RO.mixed_permeate[0.0].conc_mass_phase_comp['Liq', 'NaCl'].unfix()

# Fix the new A & B values. This will fix them at the solved values from the previous steps
m.fs.RO.A_comp.fix()
m.fs.RO.B_comp.fix()

## Now fix the new feed flow rate and pressure loss

In [32]:
# Fix the new flow rate
m.fs.RO.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(feed_flow)
# Fix the pressure drop
m.fs.RO.deltaP.fix(pressure_loss)

print('\n')
print("DOF = ", degrees_of_freedom(m))



DOF =  -1


## We have too few degrees of Freedom. Unfix the spacer porosity variable

In [33]:
# Unfix the spacer porosity
m.fs.RO.feed_side.spacer_porosity.unfix()

print('\n')
print("DOF = ", degrees_of_freedom(m))



DOF =  0


In [34]:
results = solve(m)


--------- SOLVING ---------


--------- OPTIMAL SOLVE!!! ---------

Water Perm          4.173     l/bar/h/m**2
Salt Perm           0.082     l/h/m**2  
Porosity            0.612     dimensionless




### We could then repeat these steps to get a more accurate prediction for the membrane parameters if desired
##
##

# Part 3: Estimate membrane parameters in more complex systems

<p style="text-align: center"><img src="assets/Two_Stage_RO.png" width="40%">

In [35]:
from idaes.models.unit_models import Product, Feed
from idaes.models.unit_models.mixer import (
    Mixer,
    MomentumMixingType,
    MaterialBalanceType,
)
from idaes.core.util.initialization import propagate_state
from pyomo.environ import TransformationFactory
from pyomo.network import Arc

## Begin setting up model. This time we will use feed, product, and disposal blocks

In [36]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

m.fs.properties = NaClParameterBlock()
m.fs.feed = Feed(property_package=m.fs.properties)
m.fs.product = Product(property_package=m.fs.properties)
m.fs.disposal = Product(property_package=m.fs.properties)

## Define the two RO Units and the Permeate Mixer

In [37]:
m.fs.RO_1 = ReverseOsmosis0D(
        property_package=m.fs.properties,
        has_pressure_change=True,
        pressure_change_type=PressureChangeType.calculated,
        mass_transfer_coefficient=MassTransferCoefficient.calculated,
        concentration_polarization_type=ConcentrationPolarizationType.calculated,
        has_full_reporting = True
        )

m.fs.RO_2 = ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            pressure_change_type=PressureChangeType.calculated,
            mass_transfer_coefficient=MassTransferCoefficient.calculated,
            concentration_polarization_type=ConcentrationPolarizationType.calculated,
            has_full_reporting = True
            )

m.fs.permeate_mixer = Mixer(
        property_package=m.fs.properties,
        has_holdup=False,
        num_inlets = 2,
    )

## Connect unit models

In [38]:
# Connect the feed to the 1st RO Unit
m.fs.feed_to_RO_1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.RO_1.inlet,
    )

# Connect the 1st RO Unit Retentate to the 2nd RO Unit
m.fs.RO_1_to_RO_2 = Arc(
        source=m.fs.RO_1.retentate,
        destination=m.fs.RO_2.inlet,
    )

# Connect the 1nd RO Unit Permeate to the permeate mixer
m.fs.RO_1_to_permeate_mixer = Arc(
        source=m.fs.RO_1.permeate,
        destination=m.fs.permeate_mixer.inlet_1,
    )

# Connect the 2nd RO Unit Permeate to the permeate mixer
m.fs.RO_2_to_permeate_mixer = Arc(
        source=m.fs.RO_2.permeate,
        destination=m.fs.permeate_mixer.inlet_2,
    )

# Connect the permeate mixer to the product
m.fs.permeate_mixer_to_product = Arc(
        source=m.fs.permeate_mixer.outlet,
        destination=m.fs.product.inlet,
    )

m.fs.feed.properties[0.0].conc_mass_phase_comp
m.fs.feed.properties[0.0].dens_mass_phase
m.fs.feed.properties[0.0].flow_vol_phase
m.fs.product.properties[0.0].conc_mass_phase_comp
m.fs.product.properties[0.0].dens_mass_phase
m.fs.product.properties[0.0].flow_vol_phase

TransformationFactory("network.expand_arcs").apply_to(m)

## Define the operating conditions. Starting with the 4 DOF for the feed 

## Degrees of Freedom

#### 4-DOF: Inlet feed state variables (i.e. temperature, pressure, component flowrates)

#### 7-DOF: RO Unit 1

#### 7-DOF: RO Unit 2



In [39]:
print("DOF = ", degrees_of_freedom(m))

DOF =  18


In [40]:
m.fs.feed.properties[0].flow_mass_phase_comp['Liq', 'NaCl'].fix(0.035)  # mass flow rate of NaCl (kg/s)
m.fs.feed.properties[0].flow_mass_phase_comp['Liq', 'H2O'].fix(0.965)   # mass flow rate of water (kg/s)
m.fs.feed.properties[0].pressure.fix(50e5)                              # feed pressure (Pa)           
m.fs.feed.properties[0].temperature.fix(298.15)                         # feed temperature (K)

In [41]:
m.fs.RO_1.area.fix(20)                                              # membrane area (m^2)
m.fs.RO_1.A_comp.fix(4.2e-12)                                       # membrane water permeability (m/Pa/s)
m.fs.RO_1.B_comp.fix(3.5e-8)                                        # membrane salt permeability (m/s)
m.fs.RO_1.permeate.pressure[0].fix(101325)                          # permeate pressure (Pa)
m.fs.RO_1.feed_side.channel_height.fix(1e-3)                        # channel height in membrane stage (m)
m.fs.RO_1.feed_side.spacer_porosity.fix(0.8)                        # spacer porosity in membrane stage (-)         
m.fs.RO_1.length.fix(1.016)                                         # membrane length (m)

In [42]:
m.fs.RO_2.area.fix(10)                                              # membrane area (m^2)
m.fs.RO_2.A_comp.fix(4.2e-12)                                       # membrane water permeability (m/Pa/s)
m.fs.RO_2.B_comp.fix(3.5e-8)                                        # membrane salt permeability (m/s)
m.fs.RO_2.permeate.pressure[0].fix(101325)                          # permeate pressure (Pa)
m.fs.RO_2.feed_side.channel_height.fix(1e-3)                        # channel height in membrane stage (m)
m.fs.RO_2.feed_side.spacer_porosity.fix(0.8)                        # spacer porosity in membrane stage (-)         
m.fs.RO_2.length.fix(1.016)                                         # membrane length (m)

In [43]:
print("DOF = ", degrees_of_freedom(m))
assert_no_degrees_of_freedom(m)

DOF =  0


## Scale both units

In [44]:
set_scaling_factor(m.fs.RO_1.area, 1e-2)
set_scaling_factor(m.fs.RO_1.feed_side.area, 1e-3)
set_scaling_factor(m.fs.RO_1.width, 1e-2)
set_scaling_factor(m.fs.RO_2.area, 1e-2)
set_scaling_factor(m.fs.RO_2.feed_side.area, 1e-3)
set_scaling_factor(m.fs.RO_2.width, 1e-2)

m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))
calculate_scaling_factors(m) 

### Initialize the Feed and pass the results to the 1st RO Unit

In [45]:
m.fs.feed.initialize()
propagate_state(m.fs.feed_to_RO_1)

2024-05-17 15:28:14 [INFO] idaes.init.fs.feed: Initialization Complete.


### Initialize 1st RO Unit and pass the results to the 2nd RO Unit and Permeate Mixer

In [46]:
m.fs.RO_1.initialize()
propagate_state(m.fs.RO_1_to_RO_2)
propagate_state(m.fs.RO_1_to_permeate_mixer)

2024-05-17 15:28:14 [INFO] idaes.init.fs.RO_1.feed_side: Initialization Complete
2024-05-17 15:28:15 [INFO] idaes.init.fs.RO_1: Initialization Complete: optimal - Optimal Solution Found


### Initialize 2nd RO Unit and pass the results to the Permeate Mixer

In [47]:
m.fs.RO_2.initialize()
propagate_state(m.fs.RO_2_to_permeate_mixer)

2024-05-17 15:28:15 [INFO] idaes.init.fs.RO_2.feed_side: Initialization Complete
2024-05-17 15:28:15 [INFO] idaes.init.fs.RO_2: Initialization Complete: optimal - Optimal Solution Found


### Initialize Permeate Mixer and pass the results to the Product

In [48]:
m.fs.permeate_mixer.initialize()
propagate_state(m.fs.permeate_mixer_to_product)

2024-05-17 15:28:15 [INFO] idaes.init.fs.permeate_mixer: Initialization Complete: optimal - Optimal Solution Found


### Initialize Product

In [49]:
m.fs.product.initialize()

2024-05-17 15:28:15 [INFO] idaes.init.fs.product: Initialization Complete.


## Solve the fully initialized system

In [50]:
solver = get_solver()
print("\n--------- SOLVING ---------\n")
results = solver.solve(m)
if check_optimal_termination(results):
    print("\n--------- OPTIMAL SOLVE!!! ---------\n")


--------- SOLVING ---------


--------- OPTIMAL SOLVE!!! ---------



## Now let's look at how we would solve the permeability of both RO Units simultaneously

In [51]:
def solve_A(m):

    # Start by freeing up the DOF by unfixing the A values for both units
    m.fs.RO_1.A_comp.unfix()
    m.fs.RO_2.A_comp.unfix()

    # Now we can fix the permeate flowrate 
    m.fs.product.properties[0.0].flow_mass_phase_comp['Liq', 'H2O'].fix(0.1)
    
    # We can create a custom constraint to ensure that the A values are the same
    m.fs.eq_water_perm_similarity = Constraint(
        expr=m.fs.RO_1.A_comp[0,"H2O"]
        == m.fs.RO_2.A_comp[0,"H2O"]
    )
    
    solver = get_solver()
    print("\n--------- SOLVING FOR A ---------\n")
    results = solver.solve(m)
    if check_optimal_termination(results):
        print("\n--------- OPTIMAL SOLVE!!! ---------\n")

In [52]:
def solve_B(m):
    # Start by freeing up the DOF by unfixing the B values for both units
    m.fs.RO_1.B_comp.unfix()
    m.fs.RO_2.B_comp.unfix()

    # Now we can fix the salt concentration in the permeate
    m.fs.product.properties[0.0].conc_mass_phase_comp['Liq', 'NaCl'].fix(200 * pyunits.mg/pyunits.L)

    # We can create a custom constraint to ensure that the B values are the same
    m.fs.eq_salt_perm_similarity = Constraint(
        expr=m.fs.RO_1.B_comp[0,"NaCl"]
        == m.fs.RO_2.B_comp[0,"NaCl"]
    )

    solver = get_solver()
    print("\n--------- SOLVING FOR B ---------\n")
    results = solver.solve(m)
    if check_optimal_termination(results):
        print("\n--------- OPTIMAL SOLVE!!! ---------\n")

## Membrane parameters before fitting:

In [53]:
print(f'{"Param":<20s}{"RO1":<10s}{"RO2":<10s}{"Units":<10s}')
print(f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}')
print(f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}')
print(f'{"Porosity":<20s}{value(m.fs.RO_1.feed_side.spacer_porosity):<10.3f}{value(m.fs.RO_2.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO_1.feed_side.spacer_porosity)):<10s}')

Param               RO1       RO2       Units     
Water Perm          1.512     1.512     l/bar/h/m**2
Salt Perm           0.126     0.126     l/h/m**2  
Porosity            0.800     0.800     dimensionless


## Membrane parameters after fitting A:

In [54]:
solve_A(m)
print(f'{"Param":<20s}{"RO1":<10s}{"RO2":<10s}{"Units":<10s}')
print(f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}')
print(f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}')
print(f'{"Porosity":<20s}{value(m.fs.RO_1.feed_side.spacer_porosity):<10.3f}{value(m.fs.RO_2.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO_1.feed_side.spacer_porosity)):<10s}')


--------- SOLVING FOR A ---------


--------- OPTIMAL SOLVE!!! ---------

Param               RO1       RO2       Units     
Water Perm          0.747     0.747     l/bar/h/m**2
Salt Perm           0.126     0.126     l/h/m**2  
Porosity            0.800     0.800     dimensionless


## Membrane parameters after fitting B:

In [55]:
solve_B(m)
print(f'{"Param":<20s}{"RO1":<10s}{"RO2":<10s}{"Units":<10s}')
print(f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}')
print(f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}')
print(f'{"Porosity":<20s}{value(m.fs.RO_1.feed_side.spacer_porosity):<10.3f}{value(m.fs.RO_2.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO_1.feed_side.spacer_porosity)):<10s}')


--------- SOLVING FOR B ---------


--------- OPTIMAL SOLVE!!! ---------

Param               RO1       RO2       Units     
Water Perm          0.757     0.757     l/bar/h/m**2
Salt Perm           0.059     0.059     l/h/m**2  
Porosity            0.800     0.800     dimensionless


In [56]:
print('\n')
print(f'{"Feed Flow Rate":<20s}{value(m.fs.feed.flow_mass_phase_comp[0 ,"Liq", "H2O"]):<10.3f}{str(pyunits.get_units(m.fs.feed.flow_mass_phase_comp[0 ,"Liq", "H2O"])):<10s}')
print(f'{"Feed Conc":<20s}{value(m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"]):<10.3f}{str(pyunits.get_units(m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"])):<10s}')
print(f'{"Feed Pressure":<20s}{value(pyunits.convert(m.fs.feed.properties[0].pressure, to_units=pyunits.bar)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.feed.properties[0].pressure, to_units=pyunits.bar))):<10s}')
print('\n')
print(f'{"RO1 Area":<20s}{value(m.fs.RO_1.area):<10.3f}{str(pyunits.get_units(m.fs.RO_1.area)):<10s}')
print(f'{"RO2 Area":<20s}{value(m.fs.RO_2.area):<10.3f}{str(pyunits.get_units(m.fs.RO_2.area)):<10s}')
print('\n')
print(f'{"RO1 Perm Flow":<20s}{value(value(pyunits.convert(m.fs.RO_1.mixed_permeate[0.0].flow_vol_phase["Liq"], to_units=pyunits.L * pyunits.s ** -1))):<10.3f}{str(pyunits.get_units(pyunits.L * pyunits.s ** -1)):<10s}')
print(f'{"RO2 Perm Flow":<20s}{value(value(pyunits.convert(m.fs.RO_2.mixed_permeate[0.0].flow_vol_phase["Liq"], to_units=pyunits.L * pyunits.s ** -1))):<10.3f}{str(pyunits.get_units(pyunits.L * pyunits.s ** -1)):<10s}')
print('\n')
print(f'{"Total Perm Flow":<20s}{value(pyunits.convert(m.fs.product.properties[0].flow_vol_phase["Liq"], to_units=pyunits.L * pyunits.s ** -1)):<10.3f}{str(pyunits.get_units(pyunits.L * pyunits.s ** -1)):<10s}')
print(f'{"Perm Conc":<20s}{value(pyunits.convert(m.fs.product.properties[0].conc_mass_phase_comp["Liq", "NaCl"], to_units=pyunits.g * pyunits.L ** -1)):<10.3f}{str(pyunits.get_units(pyunits.g * pyunits.L ** -1)):<10s}')
print('\n')
print(f'{"Param":<20s}{"RO1":<10s}{"RO2":<10s}{"Units":<10s}')
print(f'{"Water Perm":<20s}{value(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.A_comp[0,"H2O"], to_units=pyunits.L * pyunits.m**-2 * pyunits.bar **-1 * pyunits.hr ** -1))):<10s}')
print(f'{"Salt Perm":<20s}{value(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{value(pyunits.convert(m.fs.RO_2.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1)):<10.3f}{str(pyunits.get_units(pyunits.convert(m.fs.RO_1.B_comp[0,"NaCl"], to_units=pyunits.L * pyunits.m**-2 * pyunits.hr ** -1))):<10s}')
print(f'{"Porosity":<20s}{value(m.fs.RO_1.feed_side.spacer_porosity):<10.3f}{value(m.fs.RO_2.feed_side.spacer_porosity):<10.3f}{str(pyunits.get_units(m.fs.RO_1.feed_side.spacer_porosity)):<10s}')
print('\n')



Feed Flow Rate      0.965     kg/s      
Feed Conc           35.751    kg/m**3   
Feed Pressure       50.000    bar       


RO1 Area            20.000    m**2      
RO2 Area            10.000    m**2      


RO1 Perm Flow       0.068     l/s       
RO2 Perm Flow       0.032     l/s       


Total Perm Flow     0.101     l/s       
Perm Conc           0.200     g/l       


Param               RO1       RO2       Units     
Water Perm          0.757     0.757     l/bar/h/m**2
Salt Perm           0.059     0.059     l/h/m**2  
Porosity            0.800     0.800     dimensionless


